In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage
import os

import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow

import cv2

In [ ]:
import tensorflow.keras.applications as model_zoo

BaseModel = model_zoo.MobileNetV3Small(include_top=False,
                                        weights='imagenet',
                                        alpha=0.75,
                                        pooling="avg")
BaseModel.trainable = False # freeze pretrained weights
BaseModel.training=False # set to inference mode


In [ ]:
rootDirPath = "/content/drive/MyDrive/CoronaryArtery"
defectPath = os.path.join(rootDirPath, "Defect")
normalPath = os.path.join(rootDirPath, "Ordinary")

In [ ]:
image = Image.open(os.path.join(defectPath, "image_1.jpg"))
imageArray = np.array(image)
print(imageArray.shape)

In [ ]:
imageArrayExpand = np.expand_dims(imageArray, axis=0)
embedding = BaseModel(imageArrayExpand, training=False)
print(embedding.shape)

In [ ]:
def CreateBinaryData():
  """
  1: class WITH coronary artery defects
  0: class WITH NO coronary artery defect
  """

  y_train = []
  x_train = []

  for pathTrue, pathFalse in zip(os.listdir(defectPath), os.listdir(normalPath)):

    imageTrue = np.array(Image.open(os.path.join(defectPath, pathTrue)))
    imageFalse = np.array(Image.open(os.path.join(normalPath, pathFalse)).convert("RGB"))

    imageTrue = np.expand_dims(imageTrue, axis=0)
    imageFalse = np.expand_dims(imageFalse, axis=0)

    imageTrue = BaseModel(imageTrue, training=False)[0]
    imageFalse = BaseModel(imageFalse, training=False)[0]

    x_train.append(imageTrue)
    x_train.append(imageFalse)

    y_train.append(1)
    y_train.append(0)

  x_train = np.array(x_train)
  y_train = np.array(y_train)

  return x_train, y_train

In [ ]:
x_train, y_train = CreateBinaryData()

In [ ]:
print(x_train.shape, y_train.shape)

## BINARY CLASSIFICATION


In [ ]:
BinaryClassificationModel = tf.keras.Sequential()
input_layer = tf.keras.layers.Input(shape=(432))
output_layer = tf.keras.layers.Dense(1, activation="sigmoid")
BinaryClassificationModel.add(input_layer)
BinaryClassificationModel.add(output_layer)
BinaryClassificationModel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = BinaryClassificationModel.fit(x_train, y_train, epochs=10)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].set_title("Độ chính xác")
axes[0].plot(history.history["accuracy"])
axes[1].set_title("Hàm mất mát")
axes[1].plot(history.history["loss"])

In [ ]:
BinaryClassificationModel.save('/content/drive/MyDrive/BinaryClassificationModel.h5')

In [ ]:
InferModel = tf.keras.Sequential()

InferModel.add(BaseModel)
InferModel.add(BinaryClassificationModel)

## BOUNDING BOX REGRESSION

In [ ]:
def CreateDataForBBox():
  x_train_bbox = []
  y_train_bbox = []

  for pathTrue in os.listdir(defectPath):
    imageTrue = np.array(Image.open(os.path.join(defectPath, pathTrue)))
    imageTrue = np.expand_dims(imageTrue, axis=0)
    imageTrue = BaseModel(imageTrue, training=False)[0]

    x_train_bbox.append(imageTrue)

  # [ymin, xmin, ymax, xmax]
  y_load = np.load("/content/drive/MyDrive/BBoxCoordinates.npy")

  for i in range(y_load.shape[0]):
    for j in range(y_load.shape[1]):
      a = np.array(y_load[i][j])
      a *= 512
      a = np.round(a)
      print(a)
      y_train_bbox.append(a)

  x_train_bbox = np.array(x_train_bbox)
  y_train_bbox = np.array(y_train_bbox)

  return x_train_bbox, y_train_bbox

In [ ]:
x_train_bbox, y_train_bbox = CreateDataForBBox()

print(x_train_bbox.shape, y_train_bbox.shape)

In [ ]:
ModelRegression = tf.keras.models.Sequential()
input_layer_reg = tf.keras.layers.Input(shape=(432))
output_layer_reg = tf.keras.layers.Dense(4, activation="linear")
ModelRegression.add(input_layer_reg)
ModelRegression.add(output_layer_reg)
# compile the model with an optimizer and loss function
ModelRegression.compile(optimizer='adam', loss='mae', metrics=['mse'])
ModelRegression.summary()

In [ ]:
historyRegression = ModelRegression.fit(x_train_bbox, y_train_bbox, epochs=300)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].set_title("Độ lệch MAE")
axes[0].plot(historyRegression.history["mse"])
axes[1].set_title("Độ lệch MSE")
axes[1].plot(historyRegression.history["mae"])

In [ ]:
RegressionModel = tf.keras.models.Sequential()
RegressionModel.add(BaseModel)
RegressionModel.add(ModelRegression)

In [ ]:
image = cv2.imread(os.path.join(defectPath, "image_1.jpg"))
imageArray = np.array(image)
imageArrayExpand = np.expand_dims(imageArray, axis=0)
predictions = RegressionModel.predict(imageArrayExpand)

In [ ]:
predictions = np.array(predictions[0])
ymin, xmin, ymax, xmax = predictions
ymin = round(ymin)
xmin = round(xmin)
ymax = round(ymax)
xmax = round(xmax)

print(ymin, xmin, ymax, xmax)

In [ ]:
from google.colab.patches import cv2_imshow
cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

# Display the image with the bounding box
cv2_imshow(image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
ModelRegression.save("/content/drive/MyDrive/ModelRegression.h5")